In [ ]:
import numpy as np
from itertools import permutations
import matplotlib.pyplot as plt
import matplotlib.pyplot as mpl
import scipy.io
import sys
sys.path.insert(0,'..')
from matplotlib import gridspec

from analysis.summary_parallel import *
from analysis.analysis_utils import *
%matplotlib inline

## Load data

In [ ]:
my_data = np.load('../data/phase_order_model.npy')
pltcolors, pltcolors_alt = steffiscolours()
perms= np.array([[3,1,2],[1,3,2],[3,2,1],[2,3,1],[1,2,3],[2,1,3]])
num_N= len(my_data)
print("data length = " + str(num_N))

## Compare predictions of data to permutation

In [ ]:
num_correct=0
for dat in my_data:
    if np.isclose(dat[0],get_phase_order(dat[1],dat[2])[0]):
        num_correct+=1
pred = num_correct/num_N
print("perc. correct = " + str(pred))

In [ ]:
%%time

do_analysis=False
if do_analysis:
    #1000 iterations take around an hour
    number_permutations=1000
    scores = []
    sh_data=np.copy(my_data)
    for perm in range(number_permutations):
        num_correct=0
        sh_ind1=np.random.rand(num_N).argsort()
        sh_data[:,1]=sh_data[sh_ind1,1]
        
        sh_ind2=np.random.rand(num_N).argsort()
        sh_data[:,2]=sh_data[sh_ind2,2]

        for dat in sh_data:
            if np.isclose(dat[0],get_phase_order(dat[1],dat[2])[0]):
                num_correct+=1
        print("perm " +str(perm) + " perc. correct = " + str(num_correct/num_N))
        scores.append(num_correct/num_N)
    scores = np.array(scores)
    np.save("../data/permuted_data.npy", scores)
else:
    scores=np.load("../data/permuted_data.npy")

In [ ]:
with mpl.rc_context(fname="matplotlibrc"):
    fig = plt.figure(figsize = (2.8,1.4))
    spec = gridspec.GridSpec(ncols=2, nrows=1,
                         width_ratios=[3, 1], wspace=0.15,
                         hspace=0.5)
    ax1 = fig.add_subplot(spec[0])
    ax2 = fig.add_subplot(spec[1])

    ax1.hist(np.array(scores), bins = 24, color = pltcolors_alt[1], label ='permutations')
    ax2.axvline(pred, color ='firebrick', label ='data')
    ax1.set_ylabel("counts")
    ax1.set_xlabel("prop. predicted correctly")

    plt.legend(bbox_to_anchor = (0.9,1.1))
    ax2.spines.left.set_visible(False)
    ax2.set_xlim(.55,.58)
    ax2.set_yticks([])
    ax1.set_xlim(0.2,0.25)
    ax1.set_xticks([0.2,0.22,0.24])
    ax2.set_xticks([0.56,0.58])

    d = .5  
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax1.plot([1],[0], transform=ax1.transAxes, **kwargs)
    ax2.plot([-.05], [0], transform=ax2.transAxes, **kwargs)
    plt.savefig("../figures/model_perm.pdf")


### Shuffle either ISI or Freq

In [ ]:
%%time
# Shuffle frequency

number_permutations=1000
scores = []
sh_data=np.copy(my_data)
for perm in range(number_permutations):
    num_correct=0
    sh_ind=np.random.rand(num_N).argsort()
    sh_data[:,1]=sh_data[sh_ind,1]
    for dat in sh_data:
        if np.isclose(dat[0],get_phase_order(dat[1],dat[2])[0]):
            num_correct+=1
    print("perm " +str(perm) + " perc. correct = " + str(num_correct/num_N))
    scores.append(num_correct/num_N)

In [ ]:
%%time
#Shuffle ISI

number_permutations=1000
scores_ISIshuff = []
sh_data=np.copy(my_data)
for perm in range(number_permutations):
    num_correct=0
    sh_ind=np.random.rand(num_N).argsort()
    sh_data[:,2]=sh_data[sh_ind,2]
    for dat in sh_data:
        if np.isclose(dat[0],get_phase_order(dat[1],dat[2])[0]):
            num_correct+=1
    print("perm " +str(perm) + " perc. correct = " + str(num_correct/num_N))
    scores_ISIshuff.append(num_correct/num_N)

In [ ]:
with mpl.rc_context(fname="matplotlibrc"):
    fig,ax=plt.subplots(2,figsize = (8,3))
    ax[0].hist(np.array(scores), bins = 24, color = pltcolors_alt[1], label ='permutations')
    ax[0].axvline(pred, color ='red', label ='data')
    ax[1].hist(np.array(scores_ISIshuff), bins = 24, color = pltcolors_alt[1], label ='permutations')
    ax[1].axvline(pred, color ='red', label ='data')
    ax[0].set_ylabel("counts")
    ax[1].set_ylabel("counts")

    ax[1].set_xlabel("prop. predicted correctly")
    ax[0].set_xticks(np.arange(.1,.61,.1), 
               labels = [])
    ax[1].set_xticks(np.arange(.1,.61,.1), 
               labels = ["{:.2f}".format(np.arange(.1,.61,.1)[i]) for i in range(6)])
    ax[0].legend(bbox_to_anchor = (0.9,1.1))
    plt.savefig("../figures/pred.png", facecolor = 'white')
